In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Notebook author:** emeinhardt@ucsd.edu

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span><ul class="toc-item"><li><span><a href="#Requirements" data-toc-modified-id="Requirements-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Requirements</a></span></li><li><span><a href="#Usage" data-toc-modified-id="Usage-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Usage</a></span></li></ul></li><li><span><a href="#Parameters" data-toc-modified-id="Parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Imports-/-load-data" data-toc-modified-id="Imports-/-load-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Imports / load data</a></span><ul class="toc-item"><li><span><a href="#Load/extract-sanity-checking-data" data-toc-modified-id="Load/extract-sanity-checking-data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Load/extract sanity-checking data</a></span><ul class="toc-item"><li><span><a href="#Segmental-lexicon,-prefixes,-inventory,-and-triphones" data-toc-modified-id="Segmental-lexicon,-prefixes,-inventory,-and-triphones-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Segmental lexicon, prefixes, inventory, and triphones</a></span></li><li><span><a href="#Triphone-channel-distribution-and-channel-alphabet" data-toc-modified-id="Triphone-channel-distribution-and-channel-alphabet-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Triphone channel distribution and channel alphabet</a></span></li><li><span><a href="#Corpus-contexts" data-toc-modified-id="Corpus-contexts-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Corpus contexts</a></span></li><li><span><a href="#Conversion-to-one-hot-vectors-/-sequences-thereof" data-toc-modified-id="Conversion-to-one-hot-vectors-/-sequences-thereof-3.1.4"><span class="toc-item-num">3.1.4&nbsp;&nbsp;</span>Conversion to one-hot vectors / sequences thereof</a></span></li></ul></li><li><span><a href="#Load-segmental-sequence-channel-matrices-$p(Y_0^f|X_0^k)$" data-toc-modified-id="Load-segmental-sequence-channel-matrices-$p(Y_0^f|X_0^k)$-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Load segmental sequence channel matrices $p(Y_0^f|X_0^k)$</a></span></li><li><span><a href="#Load-contextual-distribution-on-segmental-wordforms-$p(W|C)$" data-toc-modified-id="Load-contextual-distribution-on-segmental-wordforms-$p(W|C)$-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Load contextual distribution on segmental wordforms $p(W|C)$</a></span></li><li><span><a href="#Load-lexicon-metadata" data-toc-modified-id="Load-lexicon-metadata-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Load lexicon metadata</a></span></li></ul></li><li><span><a href="#Sanity-checking-calculations" data-toc-modified-id="Sanity-checking-calculations-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Sanity-checking calculations</a></span><ul class="toc-item"><li><span><a href="#$p(Y_0^i|X_0^k)$" data-toc-modified-id="$p(Y_0^i|X_0^k)$-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>$p(Y_0^i|X_0^k)$</a></span></li><li><span><a href="#$p(X_0^f|C)$-(or-$p(X_0^k|C)$)" data-toc-modified-id="$p(X_0^f|C)$-(or-$p(X_0^k|C)$)-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>$p(X_0^f|C)$ (or $p(X_0^k|C)$)</a></span></li><li><span><a href="#$p(Y_0^i)$" data-toc-modified-id="$p(Y_0^i)$-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>$p(Y_0^i)$</a></span></li><li><span><a href="#$p(Y_0^i|x_0^k)$-sampler" data-toc-modified-id="$p(Y_0^i|x_0^k)$-sampler-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>$p(Y_0^i|x_0^k)$ sampler</a></span></li><li><span><a href="#$p(\widehat{X_0^f}|Y_0^{i})$" data-toc-modified-id="$p(\widehat{X_0^f}|Y_0^{i})$-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>$p(\widehat{X_0^f}|Y_0^{i})$</a></span></li><li><span><a href="#$\widehat{p}(\widehat{X_0^f}|X_0^i;X_{i+1})$" data-toc-modified-id="$\widehat{p}(\widehat{X_0^f}|X_0^i;X_{i+1})$-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>$\widehat{p}(\widehat{X_0^f}|X_0^i;X_{i+1})$</a></span></li></ul></li><li><span><a href="#Calculation" data-toc-modified-id="Calculation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Calculation</a></span><ul class="toc-item"><li><span><a href="#Channel-sampler" data-toc-modified-id="Channel-sampler-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Channel sampler</a></span></li><li><span><a href="#Sample-parallel-posterior-estimator" data-toc-modified-id="Sample-parallel-posterior-estimator-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Sample-parallel posterior estimator</a></span></li></ul></li></ul></div>

# Overview

Given a choice of parameters $\epsilon$ and $n$, and given
 - wordform channel matrices $p(Y_0^f|X_0^f)$
 - a contextual distribution on segmental wordforms $p(X_0^f|C)$
 - segmental lexicon metadata pre-calculating $k$-cousins/$k$-spheres up to $k=4$
 
Calculate

$$\hat{p}(\hat{X}_0^f = x_0^{'f}|X_0^f = x_0^{*f}, c) = \frac{1}{n} \sum\limits_{y_0^f \in S} p(\hat{X}_0^f = x_0^{'f}|y_0^f, c)$$
 where 
  - edit distance $d(x_0^{'f}, x_0^{*f}) \leq 4$
  - $S = $ a set of $n$ samples from $p(Y_0^f|x_0^{*f})$. In practice an $n \approx 50$ seems to result in estimates that are within $10^{-6}$ of the true estimate. 
  - $p(\hat{X}_0^f = x_0^{'f}|Y_0^f = y_0^f, c) = \frac{p(y_0^f|x_0^{'f})p(x_0^{'f}|c)}{p(y_0^f | c)}$
  - $p(y_0^f| c) = \sum\limits_{v', x_0^{''f}} p(y_0^f|x_0^{''f})p(x_0^{''f}|v')p(v'|c) = \sum\limits_{x_0^{''f}} p(y_0^f|x_0^{''f})p(x_0^{''f}|c)$

## Requirements

#FIXME

## Usage

#FIXME

# Parameters

In [2]:
from os import getcwd, chdir, listdir, path, mkdir, makedirs

In [3]:
from boilerplate import *

In [4]:
repo_dir = getcwd(); repo_dir

'/mnt/cube/home/AD/emeinhar/wr'

In [5]:
# Parameters

#p(Y_0^f|X_0^k) - from step 4d
c = ''
# c = 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/LTR_Buckeye_aligned_CM_filtered_LM_filtered_CMs_by_length_by_prefix_index.pickle'
c = 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_CMs_by_length_by_prefix_index.pickle'

#p(X_0^f|C) - from step 4c
w = ''
# w = 'LD_Fisher_vocab_in_Buckeye_contexts/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_buckeye_contexts.pW_C.npy'
w = 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_C.npy'

# LTR metadata directory - contains pre-computed k-cousin/k-sphere information
m = ''
# m = 'LTR_Buckeye_aligned_w_GD_AmE_destressed'
m = 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed'

# output filepath prefix for pW_WC
o = ''
# o = 'LD_Fisher_vocab_in_Buckeye_contexts/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_buckeye_contexts.pW_WC'
o = 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_WC'

# for sanity checking...

# the sorted prefixes of W here correspond to the W dimension of file 'w'
p = ''
# p = 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.json'
p = 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.pW_V.json'

# the sorted conditioning triphones and outcome response uniphones correspond to
# the dimensions of file 'c'
t = ''
# t = 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json'
t = 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json'

# the sorted projected contexts here correspond to the C dimension of file 'w'
s = ''
# s = 'LD_Fisher_vocab_in_Buckeye_contexts/LM_filtered_buckeye_contexts.txt'
s = 'LD_Fisher_vocab_in_swbd2003_contexts/LM_filtered_swbd2003_contexts.txt'

# calculation parameters
n = ''
n = '50'

k = ''
k = '2'

r = ''
r = 'True' #if 'False', only calculate p(\hat{W}|W = w, c), i.e. don't calculate p(\hat{W}|P = p, c)

e = ''
e = 'False' #if r='False' and s='True', only calculate p(\hat{W} = w*| W = w*, c) ∀w ∈ W

In [6]:
output_dir = path.dirname(o)
if not path.exists(output_dir):
    print('Making output path {0}'.format(output_dir))
    makedirs(output_dir)

In [7]:
if n == '':
    n = 50
else:
    n = int(n)

In [8]:
if k == '':
    k = 2
else:
    k = int(k)

In [9]:
if r == '' or r == 'False':
    r = False
    raise Exception('Assuming only full wordforms have been produced is not supported currently.')
elif r == 'True':
    r = True
else:
    raise Exception(f"r must be one of {'','True','False'}, got '{r}' instead")

In [10]:
if not r and (e == '' or e == 'True'):
    e = True
    raise Exception('Only calculating the posterior probability of the actual (correct) full intended wordform is not supported currently.')
elif r and (e == '' or e == 'True'):
    raise Exception("e can only be True if r is False")
elif e == 'False':
    e = False
else:
    raise Exception(f"e must be one of {'','True','False'}, got '{e}'")

# Imports / load data

In [11]:
import pickle

In [12]:
import numpy as np
import torch
import sparse

In [13]:
from probdist import *

In [14]:
from tqdm import tqdm, tqdm_gui, tqdm_notebook

In [15]:
from joblib import Parallel, delayed

J = -1
BACKEND = 'multiprocessing'
# BACKEND = 'loky'
V = 10
PREFER = 'processes'
# PREFER = 'threads'

def identity(x):
    return x

def par(gen_expr):
    return Parallel(n_jobs=J, backend=BACKEND, verbose=V, prefer=PREFER)(gen_expr)

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
    if torch.cuda.device_count() > 1:
        print(torch.cuda.get_device_name(1))
        print('Memory Usage:')
        print('Allocated:', round(torch.cuda.memory_allocated(1)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_cached(1)/1024**3,1), 'GB')

Using device: cuda

GeForce GTX TITAN X
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [17]:
gpu = torch.device('cuda')
cpu = torch.device('cpu')

my_device = cpu

In [18]:
cuda_ft = torch.cuda.FloatTensor
cuda_dt = torch.cuda.DoubleTensor

ft = torch.FloatTensor
dt = torch.DoubleTensor

my_ft = ft
my_dt = dt

# torch.set_default_tensor_type(my_ft)
torch.set_default_tensor_type(my_dt)

## Load/extract sanity-checking data

We want to be able to make queries, spot checks, and sanity checks. That means we want to be able to reference
 1. the set of strings constituting segmental wordforms and prefixes
 2. the source and channel alphabets
 3. the channel distribution's conditioning triphones ∩ the triphones in the lexicon = the triphones in the lexicon
 4.  

Segmental wordforms were necessary for 
 - the lexicon metadata calculation (step 4b)
 - the contextual distribution on segmental wordforms (step 4c)
 - the definition of the segmental sequence channel matrices (step 4d)
 
What does each use as input? (Pass that to this notebook.)

Each notebook uses ...pW_V.json (or something slightly downstream of that)

### Segmental lexicon, prefixes, inventory, and triphones

In [19]:
pW_V = condDistsAsProbDists(importProbDist(p))

Extract 
 - `orthographic vocabulary`
 - `segmental vocabulary`

In [20]:
Vs = set(pW_V.keys())
Ws = union(mapValues(lambda dist: set(conditions(dist)), 
                     pW_V).values())
len(Vs)
len(Ws)

9411

9172

In [21]:
wordlengthsInclEdges = set(len(ds2t(w)) for w in Ws)
wordlengthsInclEdges
numWordsOfExactlyLength = {l:len(wordformsOfLength(l, Ws, True)) for l in wordlengthsInclEdges}
numWordsOfExactlyLength

{3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18}

{3: 5,
 4: 173,
 5: 1266,
 6: 1790,
 7: 1646,
 8: 1328,
 9: 1052,
 10: 828,
 11: 508,
 12: 329,
 13: 150,
 14: 66,
 15: 24,
 16: 6,
 18: 1}

In [22]:
wordlengthsNotIncludingEdges = {each-2 for each in wordlengthsInclEdges}
wordlengthsNotIncludingEdges

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16}

In [23]:
lengthFreqs = {l:len(wordformsAtLeastLlong(l, Ws, True)) for l in wordlengthsInclEdges}
lengthFreqs

{3: 9172,
 4: 9167,
 5: 8994,
 6: 7728,
 7: 5938,
 8: 4292,
 9: 2964,
 10: 1912,
 11: 1084,
 12: 576,
 13: 247,
 14: 97,
 15: 31,
 16: 7,
 18: 1}

Create mappings between `segmental wordforms` and `segmental prefixes`.

In [24]:
prefix_relation = set(union({(w,p) for p in getPrefixes(w)} for w in tqdm(Ws)))
len(prefix_relation)

100%|██████████| 9172/9172 [00:12<00:00, 748.21it/s] 


70555

Extract 
 - `segmental prefixes`

In [25]:
Ps = set(map(lambda pair: pair[1],
             prefix_relation))

Create 
 - `sorted` versions of the `segmental vocabulary` and `segmental prefixes`

In [26]:
Ws_t = tuple(sorted(list(Ws)))
Ps_t = tuple(sorted(list(Ps)))
num_wordforms = len(Ws_t)
num_prefixes = len(Ps_t)

num_wordforms
num_prefixes

9172

33059

Create mappings between `orthographic wordforms` and `segmental wordforms`.

In [27]:
v_to_Ws = mapValues(lambda dist: set(conditions(dist)),
                    pW_V)
V_W_relation = {(v,w) 
                for v in v_to_Ws 
                for w in v_to_Ws[v]}
w_to_Vs = {w:{v for v in Vs if (v,w) in V_W_relation}
           for w in Ws}

Extract
 - `segmental inventory`
 - `triphones` in the `segmental lexicon`

In [28]:
source_alphabet = lexiconToInventory(Ws)
Xs = source_alphabet
len(Xs)
Xs

41

{'aɪ',
 'aʊ',
 'b',
 'd',
 'dʒ',
 'eɪ',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'oʊ',
 'p',
 's',
 't',
 'tʃ',
 'u',
 'v',
 'w',
 'z',
 'æ',
 'ð',
 'ŋ',
 'ɑ',
 'ɔɪ',
 'ə',
 'ɚ',
 'ɛ',
 'ɪ',
 'ɹ',
 'ʃ',
 'ʊ',
 'ʌ',
 'ʒ',
 'θ',
 '⋉',
 '⋊'}

In [29]:
Xs_t = tuple(sorted(Xs))

In [30]:
lexicon_triphones = lexiconTo3factors(Ws)
len(lexicon_triphones)
list(lexicon_triphones)[:10]

7381

['d.ʃ.ɛ',
 'ɪ.k.w',
 'l.h.ɑ',
 'z.n.ɪ',
 't.b.aʊ',
 '⋊.æ.t',
 'æ.l.j',
 'ʌ.dʒ.m',
 'l.ɪ.g',
 'ɪ.p.i']

In [31]:
X012s = lexicon_triphones
X012s_t = tuple(sorted(list(X012s)))

In [32]:
triphs_with_LE = {triph for triph in lexicon_triphones if leftEdge in triph}
len(triphs_with_LE)

489

In [33]:
triphs_with_RE = {triph for triph in lexicon_triphones if rightEdge in triph}
len(triphs_with_RE)

510

### Triphone channel distribution and channel alphabet

In [34]:
p3Y1X012 = condDistsAsProbDists(importProbDist(t))

assert uniformOutcomes(p3Y1X012)

In [35]:
channel_triphones = conditions(p3Y1X012)
len(channel_triphones)

assert all(triph in channel_triphones for triph in lexicon_triphones)

7381

In [36]:
channel_alphabet = outcomes(p3Y1X012)
len(channel_alphabet)
channel_alphabet

38

{'aɪ',
 'aʊ',
 'b',
 'd',
 'dʒ',
 'eɪ',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'oʊ',
 'p',
 's',
 't',
 'tʃ',
 'u',
 'v',
 'w',
 'z',
 'æ',
 'ð',
 'ŋ',
 'ɑ',
 'ɔɪ',
 'ɚ',
 'ɛ',
 'ɪ',
 'ɹ',
 'ʃ',
 'ʊ',
 'ʌ',
 'ʒ',
 'θ'}

In [37]:
Y1s = channel_alphabet
Y1s_t = tuple(sorted(Y1s))

In [38]:
in_lexicon_inventory_but_not_in_channel_inventory = source_alphabet - channel_alphabet
in_lexicon_inventory_but_not_in_channel_inventory

assert in_lexicon_inventory_but_not_in_channel_inventory == {leftEdge, rightEdge} or in_lexicon_inventory_but_not_in_channel_inventory == {'ə', leftEdge, rightEdge}

{'ə', '⋉', '⋊'}

### Corpus contexts

In [39]:
Cs = importSeqs(s, list)
len(Cs)

Cs_t = tuple(sorted(Cs))
Cs_t[123:129]

assert tuple(Cs) == Cs_t

106295

('<rem> and doctor',
 '<rem> and electric',
 '<rem> and especially',
 '<rem> and even',
 '<rem> and for',
 '<rem> and he')

### Conversion to one-hot vectors / sequences thereof

In [40]:
# construct what you need to convert to/from one-hot representations

# look at segment sequence channel matrix notebook

In [41]:
Xmap = seqsToIndexMap(Xs)
XOHmap = seqsToOneHotMap(Xs)

In [42]:
X012map = seqsToIndexMap(X012s)
# X012OHs = seqMapToOneHots(X012map)
X012OHmap = seqsToOneHotMap(X012s)

In [43]:
Y1map = seqsToIndexMap(Y1s)

In [44]:
def dsToUniphoneIndices(ds, uniphoneToIndexMap):
    uniphoneSeq = ds2t(ds)
    return np.array([uniphoneToIndexMap[uniphone] for uniphone in uniphoneSeq])

def dsToUniphoneOHs(ds, uniphoneToOHmap):
    uniphoneSeq = ds2t(ds)
    return np.array([uniphoneToOHmap[uniphone] for uniphone in uniphoneSeq])

def dsToTriphoneSeq(ds):
    return dsToKfactorSequence(3, ds)

def dsToTriphoneIndices(ds, triphoneToIndexMap):
    triphoneSeq = dsToTriphoneSeq(ds)
    return np.array([triphoneToIndexMap[triphone] for triphone in triphoneSeq])

def dsToTriphoneOHs(ds, triphoneToOHmap):
    triphoneSeq = dsToTriphoneSeq(ds)
    return np.array([triphoneToOHmap[triphone] for triphone in triphoneSeq])

dsToUniphoneIndices('t.i.f.l', Xmap)
dsToUniphoneOHs('t.i.f.l', XOHmap)
dsToTriphoneSeq('t.i.f.l')
dsToTriphoneIndices('t.i.f.l', X012map)
dsToTriphoneOHs('t.i.f.l', X012OHmap)
dsToTriphoneOHs('t.i.f.l', X012OHmap).shape
dsToTriphoneOHs('t.i.f.l', X012OHmap)[0].shape
dsToTriphoneOHs('t.i.f.l', X012OHmap)[0][5528]
dsToTriphoneOHs('t.i.f.l', X012OHmap)[1][5352]

array([18,  9,  6, 12])

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.]])

('t.i.f', 'i.f.l')

array([3740, 1441])

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

(2, 7381)

(7381,)

0.0

0.0

In [45]:
# Y1s_RE = outcomes(p3Y1X01)
# len(Y1s_RE)
# Y1s_RE_list = sorted(list(Y1s_RE))

# print(Y1s_RE - Y1s)

# Y1REmap = seqsToIndexMap(Y1s_RE)

# Y1REOHs = seqMapToOneHots(Y1REmap)
# Y1REOHmap = seqsToOneHotMap(Y1s_RE)
Y1OHmap = seqsToOneHotMap(Y1s)
# OHY1REmap = oneHotToSeqMap(Y1s_RE)
OHY1map = oneHotToSeqMap(Y1s)

def ymap(y):
#     return Y1REmap[y]
    return Y1map[y]

def y0kMap(y0k):
    return np.array(list(map(ymap, ds2t(y0k)[1:])))

def channelSeqOHs2ds(y0k_OHs, addLeftEdge = False):
    if not addLeftEdge:
#         return t2ds(tuple( map(OHY1REmap, tuple(y0k_OHs) ) ))
        return t2ds(tuple( map(OHY1map, tuple(y0k_OHs) ) ))
#     return leftEdge + '.' + t2ds(tuple( map(OHY1REmap, tuple(y0k_OHs) ) ))
    return leftEdge + '.' + t2ds(tuple( map(OHY1map, tuple(y0k_OHs) ) ))

def channelSeqds2OHs(y0k):
    y0k_t = ds2t(y0k)
    if leftEdge == y0k_t[0]:
        y1k_t = y0k_t[1:]
    y1k_t = y0k_t[1:]
#     return np.array([Y1REOHmap[ yj ] for yj in y1k_t]) #shape should be (_, 39)
    return np.array([Y1OHmap[ yj ] for yj in y1k_t]) #shape should be (_, 38)

def y0kOHmap(y0k):
##     y0k_t = ds2t(y0k)
## #     y0k_indices = y0kMap(y0k) #np.array(list(map(lambda y1: Y1map[y1], y0k_t[1:])))
##     y1k = t2ds(y0k_t[1:])
##     y0k_OHs = dsToUniphoneOHs(y1k, Y1REOHmap)
#     return dsToUniphoneOHs(t2ds(ds2t(y0k)[1:]), Y1REOHmap)
    y0k_t = ds2t(y0k) #let l = len(y0k_t)
    if y0k_t[0] == leftEdge:
        y1k_t = y0k_t[1:]
#         return np.array([Y1REOHmap[ yj ] for yj in y1k_t]) #shape should be (l-1, 39)
        return np.array([Y1OHmap[ yj ] for yj in y1k_t]) #shape should be (l-1, 38)
#     return np.array([Y1REOHmap[ yj ] for yj in y0k_t]) #shape should be (l, 39)
    return np.array([Y1OHmap[ yj ] for yj in y0k_t]) #shape should be (l, 38)


# list(Y1s_RE)[0]
list(Y1s)[0]
# OHY1REmap( Y1REOHmap[ list(Y1s_RE)[0] ] )
OHY1map( Y1OHmap[ list(Y1s)[0] ] )

channelSeqds2OHs(leftEdge + '.'+ 't.u.p')

channelSeqOHs2ds( channelSeqds2OHs(leftEdge + '.'+ 't.u.p') , True)

channelSeqOHs2ds(y0kOHmap(leftEdge + '.' + 't.u.p'), True)
channelSeqOHs2ds(y0kOHmap('t.u.p'))
len(ds2t(leftEdge + '.' + 't.u.p'))
len(ds2t('t.u.p'))
y0kOHmap('t.u.p').shape
y0kOHmap(leftEdge + '.' + 't.u.p').shape

's'

's'

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

'⋊.t.u.p'

'⋊.t.u.p'

't.u.p'

4

3

(3, 38)

(3, 38)

In [46]:
p3Y1X012_np = condDistFamilyToNP(p3Y1X012)
p3Y1X012_np.shape

(38, 7381)

In [47]:
from random import choice

In [48]:
random_source_wordform = choice(list(Ws))
random_source_wordform

random_source_prefix = choice(list(Ps))
random_source_prefix

'⋊.g.ɑ.ɹ.n.ɪ.ʃ.⋉'

'⋊.d.ɪ.k.i'

In [49]:
def randomPrefix(l, alphabet=Xs):
    return randomString(alphabet, l, hasLeftEdge=True)

In [50]:
random_channel_prefix2 = randomPrefix(len(ds2t(random_source_wordform))-1, alphabet=Y1s)
random_channel_prefix2

'⋊.tʃ.eɪ.f.æ.aʊ.ɛ.u'

In [51]:
# random_source_prefix = getRandomKey(pX0i)
random_source_prefix = choice(list(Ps))
while ds2t(random_source_prefix)[-1] == rightEdge:
#     random_source_prefix = getRandomKey(pX0i)
    random_source_prefix = choice(list(Ps))
while len(ds2t(random_source_prefix)) > len(ds2t(random_source_wordform)):
#     random_source_prefix = getRandomKey(pX0i)
    random_source_prefix = choice(list(Ps))
random_source_prefix
random_channel_prefix = randomPrefix(len(ds2t(random_source_prefix))-1, alphabet=Y1s)
random_channel_prefix

'⋊.p.ɹ.ɑ.b.ə.b.l'

'⋊.ɔɪ.dʒ.ɪ.h.b.h.ɚ'

In [52]:
def sourcePrefixToTriphones(x0k):
    xp_t = ds2t(x0k) #"x prefix"
    
#     xi = xp_t[-2] #just-completed segment
#     xk = xp_t[-1] #upcoming segment that we only get coarticulatory information about
    
#     xik_ds = t2ds((xi, xk))
#     preview_dist = p3Y1X01[xik_ds]
    
    x012s = dsToKfactorSequence(3, t2ds(xp_t))
    return x012s

random_triphoneSeq = sourcePrefixToTriphones(random_source_prefix)
random_triphoneSeq
threeFactorSequenceToDS(random_triphoneSeq)

('⋊.p.ɹ', 'p.ɹ.ɑ', 'ɹ.ɑ.b', 'ɑ.b.ə', 'b.ə.b', 'ə.b.l')

'⋊.p.ɹ.ɑ.b.ə.b.l'

In [53]:
def sourcePrefixToTriphoneIndices(x0k):
    triphoneSequence = sourcePrefixToTriphones(x0k)
    return tuple(map(lambda x012: X012map[x012], triphoneSequence))

sourcePrefixToTriphoneIndices(random_source_prefix)

(7138, 3323, 6372, 4846, 364, 5101)

## Load segmental sequence channel matrices $p(Y_0^f|X_0^k)$

Index $i$ of `pY0f_X0ks` is a tensor of shape $|W| \times |Y_1| \times (i+1)$: for each wordform $w$ in $W$, there is a length $i+1$ stack of vectors giving the channel distribution for the length $i+1$ prefix (not including word edge symbols) of $w$.

In [54]:
pY0f_X0ks = pickle.load(open(c, 'rb'))
len(pY0f_X0ks)

15

In [55]:
pY0f_X0ks[0].dtype

dtype('float64')

In [56]:
wordlengthsNotIncludingEdges
wordlengthsInclEdges
lengthFreqs

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16}

{3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18}

{3: 9172,
 4: 9167,
 5: 8994,
 6: 7728,
 7: 5938,
 8: 4292,
 9: 2964,
 10: 1912,
 11: 1084,
 12: 576,
 13: 247,
 14: 97,
 15: 31,
 16: 7,
 18: 1}

In [57]:
len(Ws_t)
len(Y1s_t)

9172

38

In [58]:
pY0f_X0ks[0].shape
pY0f_X0ks[1].shape
pY0f_X0ks[2].shape
pY0f_X0ks[3].shape

(9172, 38, 1)

(9172, 38, 2)

(9172, 38, 3)

(9172, 38, 4)

In [59]:
pY0f_X0ks[0][0]

array([[ 0.65264711],
       [ 0.00700978],
       [ 0.02545656],
       [ 0.02545656],
       [ 0.00682531],
       [ 0.06235012],
       [ 0.00682531],
       [ 0.00700978],
       [ 0.00682531],
       [ 0.00700978],
       [ 0.0051651 ],
       [ 0.00682531],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00682531],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00682531],
       [ 0.00664084],
       [ 0.00682531],
       [ 0.00700978],
       [ 0.00682531],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00682531],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00682531],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00682531],
       [ 0.00700978]])

In [60]:
p3Y1X012_np.shape

(38, 7381)

In [61]:
def retrieve_CM_for(x0f_LERE, non_edge_prefix_length):
    return pY0f_X0ks[non_edge_prefix_length-1][Ws_t.index(x0f_LERE)]

In [62]:
w_0 = Ws_t[0]; w_0
w_0_prefix = t2ds(ds2t(Ws_t[0])[1:3+2]); w_0_prefix

pY0f_X0ks[3][0].shape
pY0f_X0ks[3][0] #assume the produced prefix is of length i+1=3+1=4 (not including word edges)

w_0_whole_prefix = t2ds(ds2t(Ws_t[0])[0:3+3]); w_0_whole_prefix
w_0_whole_prefix_3factors = sourcePrefixToTriphones(w_0_whole_prefix); w_0_whole_prefix_3factors
w_0_whole_prefix_3factor_indices = sourcePrefixToTriphoneIndices(w_0_whole_prefix); w_0_whole_prefix_3factor_indices
w_0_prefix_CM = np.vstack([p3Y1X012_np[:,source_triph_idx] for source_triph_idx in w_0_whole_prefix_3factor_indices]).T; w_0_prefix_CM
w_0_prefix_CM.shape

assert np.allclose( w_0_prefix_CM, pY0f_X0ks[3][0] )
assert np.allclose( pY0f_X0ks[3][0], retrieve_CM_for(Ws_t[0], 4))

'⋊.aɪ.d.i.l.aɪ.z.⋉'

'aɪ.d.i.l'

(38, 4)

array([[ 0.65264711,  0.02141693,  0.00703146,  0.007226  ],
       [ 0.00700978,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.02545656,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.02545656,  0.58122414,  0.00693832,  0.00713029],
       [ 0.00682531,  0.00821919,  0.00675326,  0.00694011],
       [ 0.06235012,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.00682531,  0.00833257,  0.00684642,  0.00703584],
       [ 0.00700978,  0.02552273,  0.00703146,  0.007226  ],
       [ 0.00682531,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.00855777,  0.73787478,  0.007226  ],
       [ 0.0051651 ,  0.00734595,  0.00603577,  0.00620276],
       [ 0.00682531,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.00855777,  0.00703146,  0.70851712],
       [ 0.00700978,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.01630829,  0.00703146,  0.007226  ],
       [ 0.00700978,  0.00855777,  0.00703146,  0.02155085],
       [ 0.00700978,  0.

'⋊.aɪ.d.i.l.aɪ'

('⋊.aɪ.d', 'aɪ.d.i', 'd.i.l', 'i.l.aɪ')

(6892, 5, 495, 1468)

array([[ 0.65264711,  0.02141693,  0.00703146,  0.007226  ],
       [ 0.00700978,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.02545656,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.02545656,  0.58122414,  0.00693832,  0.00713029],
       [ 0.00682531,  0.00821919,  0.00675326,  0.00694011],
       [ 0.06235012,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.00682531,  0.00833257,  0.00684642,  0.00703584],
       [ 0.00700978,  0.02552273,  0.00703146,  0.007226  ],
       [ 0.00682531,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.00855777,  0.73787478,  0.007226  ],
       [ 0.0051651 ,  0.00734595,  0.00603577,  0.00620276],
       [ 0.00682531,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.00855777,  0.00703146,  0.70851712],
       [ 0.00700978,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.01630829,  0.00703146,  0.007226  ],
       [ 0.00700978,  0.00855777,  0.00703146,  0.02155085],
       [ 0.00700978,  0.

(38, 4)

**Notes:**

 - `|pY0f_X0ks|` = `17`
 - `pY0f_X0ks[i]` = the lexical channel matrices for prefixes of length `i+3` (incl. word edges)
 - `pY0f_X0ks[i]` =  the lexical channel matrices for prefixes of length `i+1` (*not* incl. word edges)
 - `|pY0f_X0ks[i]|` = `(|W|, |Y1|, i+1)` = (segmental wordforms, Y1s, number of triphones)
 - `l ∈ [1,17]`
 - `pY0f_X0ks[l-1]` = the lexical channel matrices for prefixes of length `l+2` (incl. word edges)
 - `pY0f_X0ks[l-1]` = the lexical channel matrices for prefixes of length `l` (*not* incl. word edges)
 - `|pY0f_X0ks[l-1]|` = `(|W|, |Y1|, l)` = (segmental wordforms, Y1s, number of triphones)

## Load contextual distribution on segmental wordforms $p(W|C)$

In [63]:
pW_C = np.load(w)
pW_C.shape
pW_C.dtype
pW_C.nbytes / 1e9

(9172, 106295)

dtype('float64')

7.79950192

In [64]:
len(Ws_t)
len(Cs_t)

9172

106295

In [65]:
Cs_t[Cs_t.index('a couple of')]
pW_C[:,Cs_t.index('a couple of')]
isNormalized_np(pW_C[:,Cs_t.index('a couple of')])

'a couple of'

array([  2.52036749e-10,   1.50634197e-08,   2.52036749e-10, ...,
         5.54693797e-10,   4.15967189e-08,   6.22123621e-07])

True

In [66]:
Cs_t.index('a couple of')

1753

In [67]:
def pW_C_lookup(w=None,c=None):
    if w is None and c is None:
        raise Exception('Must specify at least one of a context string or segmental wordform string')
    if w is None:
        my_pW_c = pW_C[:,Cs_t.index(c)]
        my_pW_c_as_dict = dict(zip(Ws_t, my_pW_c))
        assert isNormalized(my_pW_c_as_dict)
        return ProbDist(my_pW_c_as_dict)
    if w is not None and c is not None:
        my_pw_c = pW_C[Ws_t.index(w), Cs_t.index(c)]
        return my_pw_c
    if c is None:
        my_pw_C = pW_C[Ws_t.index(w), :]
        my_pw_C_as_dict = dict(zip(Cs_t, my_pw_C))
        return my_pw_C_as_dict

In [68]:
my_dist = pW_C_lookup(c='a couple of')

In [69]:
my_dist.most_common(10)

[('⋊.t.aɪ.m.z.⋉', 0.70170688240694257),
 ('⋊.ð.ɛ.m.⋉', 0.058586970250377776),
 ('⋊.ð.ə.⋉', 0.054670419056498991),
 ('⋊.ʌ.ð.ɚ.⋉', 0.036645416028036247),
 ('⋊.m.aɪ.⋉', 0.028159696907432537),
 ('⋊.ð.oʊ.z.⋉', 0.020634293629461325),
 ('⋊.h.ʌ.n.d.ɹ.ə.d.⋉', 0.017489202057292059),
 ('⋊.l.aɪ.k.⋉', 0.014006091693604115),
 ('⋊.g.ʊ.d.⋉', 0.005842215226429354),
 ('⋊.m.ɪ.l.i.ə.n.⋉', 0.0053049068571393984)]

In [70]:
random_context = choice(Cs_t); random_context
Cs_t.index(random_context)
pW_C_lookup(c=random_context).most_common(10)

'being in heat'

13639

[('⋊.æ.n.d.⋉', 0.36534224693708794),
 ('⋊.l.aɪ.k.⋉', 0.18219748267984689),
 ('⋊.ɹ.aɪ.t.⋉', 0.16174375068984664),
 ('⋊.aɪ.⋉', 0.033301183238721624),
 ('⋊.j.u.⋉', 0.027389714269493486),
 ('⋊.ɪ.t.⋉', 0.025016102115418593),
 ('⋊.ʌ.p.⋉', 0.023129672521393377),
 ('⋊.w.eɪ.v.⋉', 0.021931461327811319),
 ('⋊.ɪ.n.⋉', 0.020552906885157408),
 ('⋊.ɪ.z.⋉', 0.013960712979179375)]

## Load lexicon metadata

In [71]:
cousin_fn_map = {i:'{0}cousinsOf.npz'.format(i) 
                 for i in range(5)}
sphere_fn_map = {i:'{0}spheresOf.npz'.format(i) 
                 for i in range(5)}

In [72]:
cousin_fn_map

{0: '0cousinsOf.npz',
 1: '1cousinsOf.npz',
 2: '2cousinsOf.npz',
 3: '3cousinsOf.npz',
 4: '4cousinsOf.npz'}

In [73]:
sphere_fn_map

{0: '0spheresOf.npz',
 1: '1spheresOf.npz',
 2: '2spheresOf.npz',
 3: '3spheresOf.npz',
 4: '4spheresOf.npz'}

In [74]:
assert all(fn in listdir(m) for fn in cousin_fn_map.values())
assert all(fn in listdir(m) for fn in sphere_fn_map.values())

In [75]:
chdir(m)

In [76]:
cousin_mats = mapValues(sparse.load_npz, cousin_fn_map)

In [77]:
sphere_mats = mapValues(sparse.load_npz, sphere_fn_map)

In [78]:
chdir(repo_dir)

The $k$th-cousin/sphere matrix $m$ is a matrix of shape $|P| \times |W|$, where $m_{i,j}$ = 1 iff prefix $p_i$ is in a $k$-cousin/sphere relation to wordform $w_j$.

In [79]:
len(Ps_t)
len(Ws_t)

33059

9172

In [80]:
cousin_mats
mapValues(lambda m: m.nbytes / 1e9,
          cousin_mats)

{0: <COO: shape=(33059, 9172), dtype=uint8, nnz=70555, fill_value=0>,
 1: <COO: shape=(33059, 9172), dtype=uint8, nnz=966262, fill_value=0>,
 2: <COO: shape=(33059, 9172), dtype=uint8, nnz=8845129, fill_value=0>,
 3: <COO: shape=(33059, 9172), dtype=uint8, nnz=36334546, fill_value=0>,
 4: <COO: shape=(33059, 9172), dtype=uint8, nnz=59184373, fill_value=0>}

{0: 0.001199435,
 1: 0.016426454,
 2: 0.150367193,
 3: 0.617687282,
 4: 1.006134341}

In [81]:
sphere_mats
mapValues(lambda m: m.nbytes / 1e9,
          sphere_mats)

{0: <COO: shape=(33059, 9172), dtype=uint8, nnz=9172, fill_value=0>,
 1: <COO: shape=(33059, 9172), dtype=uint8, nnz=30652, fill_value=0>,
 2: <COO: shape=(33059, 9172), dtype=uint8, nnz=370148, fill_value=0>,
 3: <COO: shape=(33059, 9172), dtype=uint8, nnz=2062370, fill_value=0>,
 4: <COO: shape=(33059, 9172), dtype=uint8, nnz=3207990, fill_value=0>}

{0: 0.000155924, 1: 0.000521084, 2: 0.006292516, 3: 0.03506029, 4: 0.05453583}

In [82]:
# c1.coords

In [83]:
c1 = cousin_mats[1]
(c1.coords[0][-2], c1.coords[1][-2])
Ps_t[c1.coords[0][-2]]
Ws_t[c1.coords[1][-2]]

(33057, 6949)

'⋊.θ.ʌ.n.d.ɚ'

'⋊.w.ʌ.n.d.ɚ.⋉'

In [84]:
m

'LTR_newdic_destressed_aligned_w_GD_AmE_destressed'

In [85]:
# segmental_wordforms = importSeqs(path.join(m, 'LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V_Transcriptions.txt'))
# len(segmental_wordforms)

In [86]:
# list(segmental_wordforms)[0]

In [87]:
# Ws_t = tuple(sorted(list(map(padInputSequenceWithBoundaries,
#                              segmental_wordforms))))
# len(Ws_t)

In [88]:
# Ps = union(map(lambda w: getPrefixes(padInputSequenceWithBoundaries(w)), segmental_wordforms))
# len(Ps)

In [89]:
# Ps_t = tuple(sorted(list(Ps)))
# len(Ps_t)

# Sanity-checking calculations

In [90]:
# FIXME update this cell once you add in cells for doing calculations the old-fashioned way 
# and verifying/demonstrating that the calculation cell works correctly

## $p(Y_0^i|X_0^k)$

In [91]:
len(pY0f_X0ks)
sorted(wordlengthsNotIncludingEdges)
assert len(pY0f_X0ks) == sorted(wordlengthsNotIncludingEdges)[-1]-1

15

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16]

In [92]:
len(Ws)
len(Ps)

9172

33059

In [93]:
# for each_l in pY0f_X0ks:
    
#     assert each_l.shape

In [94]:
pY0f_X0ks[0].shape

(9172, 38, 1)

In [95]:
def pY0iX0k(y0i, x0k, debug = False):
    xp_t = ds2t(x0k) #"x prefix"
    yp_t = ds2t(y0i) #"y prefix"
#     assert xp_t[0] == leftEdge
#     assert yp_t[0] == leftEdge
    if len(xp_t) < 3:
        raise Exception('|x0k| must be ≥ 3.')
    if len(yp_t)+1 != len(xp_t):
        raise Exception('|y0i| must = |x0k|-1.')
#     if len(yp_t) != len(xp_t):
#         raise Exception('Lengths of y0i and x0k must match.')
    # if len(xp_t) == 1:
    #     if xp_t[0] != leftEdge:
    #         raise Exception('First symbol of x0k must be left word edge symbol.')    
    #     if yp_t[0] == leftEdge:
    #         return 1.0
    #     else:
    #         return 0.0
    
#     xi = xp_t[-2] #just-completed segment
#     xk = xp_t[-1] #upcoming segment that we only get coarticulatory information about
    # yk = yp_t[-1]
    # xik_ds = t2ds((xi, xk))
    # preview_dist = p3Y1X01[xik_ds]
#     if yk not in preview_dist:
#         print('y0i: {0}\nx0k: {1}\nxi: {2}\nxk: {3}\nyk: {4}\nxik: {5}'.format(y0i, x0k, xi, xk, yk, xik_ds))
#     preview_term = preview_dist[yk]
    
#     if debug:
#         print('xi = {0}\nxk = {1}\nyk = {2}'.format(xi, xk, yk))
#         print('preview term = {0}'.format(preview_term))
    
#     x0i_t = xp_t[:-1] #prefix of intended wordform that has actually been produced
    # y0i_t = yp_t[:-1]
#     x0i_ds = t2ds(x0i_t)
#     y0i_ds = t2ds(y0i_t)

#     if debug:
#         x0i_t = xp_t[:-1] #prefix of intended wordform that has actually been produced
#         x0i_ds = t2ds(x0i_t)
#         y0i_ds = t2ds(y0i_t)
#         print('produced prefix x0i = {0}'.format(x0i_ds))
#         print('perceived prefix y0i = {0}'.format(y0i_ds))
    
#     x012s = dsToKfactorSequence(3, x0i_ds)
#     x012s = dsToKfactorSequence(3, t2ds(xp_t))
    x012s = dsToKfactorSequence(3, x0k)
    # y1s = y0i_t[1:]
    y1s = yp_t[1:]
#     pairs = list(zip(x012s, y1s))
#     produced_terms = tuple([p3Y1X012[x012][y1] for x012, y1 in pairs])
    pairs = zip(x012s, y1s)
    produced_terms = (p3Y1X012[x012][y1] for x012, y1 in pairs)
    
#     result = preview_term * prod(produced_terms)
    result = prod(produced_terms)

#     if debug:
#         print('x012s: {0}'.format(x012s))
#         print('y1s: {0}'.format(y1s))
#         print('main terms: {0}'.format(produced_terms))
#         print('end result = {0}'.format(result))
    
    if debug:
#         return result, [p3Y1X012[x012] for x012 in x012s]
        return result, np.array([[p3Y1X012[x012][k] for k in sorted(p3Y1X012[x012].keys())] for x012 in x012s]).T

    return result

In [96]:
w_0
w_0_prefix
w_0_3 = t2ds(ds2t(w_0)[:3+2]); w_0_3
w_0_4 = t2ds(ds2t(w_0)[:4+2]); w_0_4
pY0f_X0ks[3][0]

'⋊.aɪ.d.i.l.aɪ.z.⋉'

'aɪ.d.i.l'

'⋊.aɪ.d.i.l'

'⋊.aɪ.d.i.l.aɪ'

array([[ 0.65264711,  0.02141693,  0.00703146,  0.007226  ],
       [ 0.00700978,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.02545656,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.02545656,  0.58122414,  0.00693832,  0.00713029],
       [ 0.00682531,  0.00821919,  0.00675326,  0.00694011],
       [ 0.06235012,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.00682531,  0.00833257,  0.00684642,  0.00703584],
       [ 0.00700978,  0.02552273,  0.00703146,  0.007226  ],
       [ 0.00682531,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.00855777,  0.73787478,  0.007226  ],
       [ 0.0051651 ,  0.00734595,  0.00603577,  0.00620276],
       [ 0.00682531,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.00855777,  0.00703146,  0.70851712],
       [ 0.00700978,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.01630829,  0.00703146,  0.007226  ],
       [ 0.00700978,  0.00855777,  0.00703146,  0.02155085],
       [ 0.00700978,  0.

In [97]:
pY0iX0k(w_0_3, w_0_4)
pY0iX0k(w_0_3, w_0_4, True)[1]
assert np.allclose( pY0f_X0ks[3][0], pY0iX0k(w_0_3, w_0_4, True)[1] )

0.1983147788326225

array([[ 0.65264711,  0.02141693,  0.00703146,  0.007226  ],
       [ 0.00700978,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.02545656,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.02545656,  0.58122414,  0.00693832,  0.00713029],
       [ 0.00682531,  0.00821919,  0.00675326,  0.00694011],
       [ 0.06235012,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.00682531,  0.00833257,  0.00684642,  0.00703584],
       [ 0.00700978,  0.02552273,  0.00703146,  0.007226  ],
       [ 0.00682531,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.00855777,  0.73787478,  0.007226  ],
       [ 0.0051651 ,  0.00734595,  0.00603577,  0.00620276],
       [ 0.00682531,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.00855777,  0.00703146,  0.70851712],
       [ 0.00700978,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.01630829,  0.00703146,  0.007226  ],
       [ 0.00700978,  0.00855777,  0.00703146,  0.02155085],
       [ 0.00700978,  0.

In [98]:
pY0iX0k('⋊.k.ɑ.n', '⋊.k.ɑ.n.t')
pY0iX0k('⋊.k.ɑ.m', '⋊.k.ɑ.n.t')

0.25068286913932425

0.0034938697256087312

In [99]:
def trimToMatch(y0i, x0f):
    x0f_t = ds2t(x0f)
    y0i_t = ds2t(y0i)
    l = len(y0i_t)
    if len(x0f_t) < l+1:
        raise Exception('|x0f| must ≥ |y0i| + 1.')
    x0k_t = x0f_t[:l+1]
    x0k = t2ds(x0k_t)
    return x0k

def trimToLength(l, x0f, pad=False, padSymbol='?'):
    x0f_t = ds2t(x0f)
#     y0i_t = ds2t(y0i)
#     l = len(y0i_t)
    if len(x0f_t) < l and not pad:
        raise Exception('x0f must be at least as long as l.')
    if len(x0f_t) < l and pad:
        x0k_t = x0f_t[:l]
        x0k_t_padded = tuple( list(x0k_t) + [padSymbol] * (l - len(x0k_t)) )
        return t2ds(x0k_t_padded)
    else:
        x0k_t = x0f_t[:l]
        x0k = t2ds(x0k_t)
        return x0k
    
# trimToMatch('t.u.p', 't.u.p')
trimToMatch('t.u.p', 't.u.p.l')
trimToLength(3, 't.u.p')
trimToLength(3, 't.u.p.l')
trimToLength(6, 't.u.p.l', True)

't.u.p.l'

't.u.p'

't.u.p'

't.u.p.l.?.?'

In [100]:
def pY0iX0f(y0i, x0f, debug = False):
    x0f_t = ds2t(x0f)
    y0i_t = ds2t(y0i)
    l = len(y0i_t)
    if len(x0f_t) < l+1:
        raise Exception('|x0f| must ≥ |y0i| + 1')
    x0k_t = x0f_t[:l+1]
    x0k = t2ds(x0k_t)
#     print(y0i)
#     print(x0k)
    return pY0iX0k(y0i, x0k, debug)

In [101]:
random_x0f = choice(Ws_t); random_x0f
lw = len(ds2t(random_x0f))
while lw-1 <= 2:
    random_x0f = getRandomKey(pW); random_x0f
    lw = len(ds2t(random_x0f))

random_y0i = randomPrefix(choice(range(2,lw-1)), alphabet=Y1s); random_y0i
random_x0f
random_y0i
pY0iX0f(random_y0i, random_x0f)

'⋊.k.æ.s.t.ɚ.⋉'

'⋊.s.g.z.aʊ.u'

'⋊.k.æ.s.t.ɚ.⋉'

'⋊.s.g.z.aʊ.u'

3.259482567155198e-11

## $p(X_0^f|C)$ (or $p(X_0^k|C)$)

In [102]:
pW_C.shape
(len(Ws), len(Cs))
assert pW_C.shape[0] == len(Ws)
assert pW_C.shape[1] == len(Cs)

(9172, 106295)

(9172, 106295)

In [103]:
Cs_t[Cs_t.index('a couple of')]

pX0f = pW_C[:,Cs_t.index('a couple of')]

pW = pW_C_lookup(c = Cs_t[Cs_t.index('a couple of')])

'a couple of'

In [104]:
# def pX0f_C_calc(w, c):
    

## $p(Y_0^i)$

In [105]:
def pXhat0fY0i_unnormalized(xhat0f, y0i):
    likelihood = pY0iX0f(y0i, xhat0f)
    prior = pW[xhat0f]
    return likelihood * prior

def pY0i(y0i):
    l = len(ds2t(y0i))
#     x0ksWithLengthl = prefixesWithLength(l)
#     py0kx0ks = tuple(pXhat0fY0k_unnormalized(x0k, y0k) for x0k in x0ksWithLengthl)

    sufficiently_long_words = {w for w in Ws if len(ds2t(w)) >= l+1}
#     print(len(sufficiently_long_words))
    #line below is slower by about a third (pypy/kotoba)
#     sufficiently_long_words = wordformsAtLeastLlong(l, True)

    py0ix0fs = (pXhat0fY0i_unnormalized(x0f, y0i) for x0f in sufficiently_long_words)
#     def foo(x0f):
#         return pXhat0fY0k_unnormalized(x0f, y0k)
    # there's no apparent benefit to using foo + generator
#     py0kx0fs = (foo(x0f) for x0f in sufficiently_long_words)
# Parallel(n_jobs=-1, verbose=50, batch_size=4, prefer="processes", backend="multiprocessing")
#     py0kx0fs = Parallel(n_jobs=8, batch_size=4, verbose=50, prefer="threads", backend="multiprocessing")(delayed(pXhat0fY0k_unnormalized)(x0f, y0k) for x0f in sufficiently_long_words)
#     py0kx0fs = Parallel(n_jobs=8, prefer="threads")(delayed(pXhat0fY0k_unnormalized)(x0f, y0k) for x0f in sufficiently_long_words)
    return sum(py0ix0fs)

In [106]:
pY0i('⋊.k.ɑ.n.t')

2.6295126487176941e-06

In [107]:
{w for w in Ws if (w,'⋊.k.ɑ.n.t') in prefix_relation}

{'⋊.k.ɑ.n.t.n.u.ɪ.t.i.⋉',
 '⋊.k.ɑ.n.t.n.ɪ.n.t.⋉',
 '⋊.k.ɑ.n.t.æ.k.t.⋉',
 '⋊.k.ɑ.n.t.ə.m.p.l.eɪ.t.⋉',
 '⋊.k.ɑ.n.t.ɛ.k.s.t.⋉',
 '⋊.k.ɑ.n.t.ɛ.s.t.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.k.t.ɚ.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.k.t.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.s.t.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.b.j.u.ʃ.ɪ.n.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.b.æ.n.d.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.d.ɪ.k.t.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.v.ɚ.s.i.⋉'}

In [108]:
pXhat0fY0i_unnormalized('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t')

3.3159315326331652e-07

## $p(Y_0^i|x_0^k)$ sampler

In [109]:
def sample_pY0iX0k(x0k, debug = False):
    xp_t = ds2t(x0k) #"x prefix"
#     yp_t = ds2t(y0i) #"y prefix"
#     if len(yp_t) != len(xp_t):
#         raise Exception('Lengths of y0i and x0i must match.')
    if len(xp_t) == 1:
        if xp_t[0] != leftEdge:
            raise Exception('First symbol of x0k must be left word edge symbol.')
        else:
            return leftEdge
    if len(xp_t) < 3:
        raise Exception('x0k must be a triphone')
    
    # xi = xp_t[-2] #just-completed segment
    # xk = xp_t[-1] #upcoming segment that we only get coarticulatory information about
    # xik = t2ds((xi, xk))
    # yk = sampleFrom(p3Y1X01[xik])
#     yk = yp_t[-1]
#     preview_term = p3Y1X01[t2ds((xi, xk))][yk]
    
#     if debug:
#         print('xi = {0}\nxk = {1}\nyk = {2}'.format(xi, xk, yk))
#         print('preview term = {0}'.format(preview_term))
    
#     x0i_t = xp_t[:-1] #prefix of intended wordform that has actually been produced
#     y0i_t = yp_t[:-1]
#     x0i_ds = t2ds(x0i_t)
#     y0i_ds = t2ds(y0i_t)

#     if debug:
#         print('produced prefix x0i = {0}'.format(x0i_ds))
#         print('perceived prefix y0i = {0}'.format(y0i_ds))
    
    x012s = dsToKfactorSequence(3, t2ds(xp_t))
#     print('x012s: {0}'.format(x012s))
#     y1s = y0i_t[1:]
    y1s = [sampleFrom(p3Y1X012[x012]) for x012 in x012s]
#     print('y1s: {0}'.format(y1s))
#     pairs = list(zip(x012s, y1s))
#     produced_terms = tuple([p3Y1X012[x012][y1] for x012, y1 in pairs])
    y0i_t = tuple( [leftEdge] + y1s )
#     print('y0i_t = {0}'.format(y0i_t))
    y0i_ds = t2ds(y0i_t)
#     print('y0i_ds = {0}'.format(y0i_ds))
#     assert len(y0i_t) == len(x0i_t)
    # yp_t = tuple([leftEdge] + y1s + [yk])
    # y0i = t2ds(yp_t)
#     if debug:
#         print('x012s: {0}'.format(x012s))
#         print('y1s: {0}'.format(tuple(y1s)))
#         print('main terms: {0}'.format(produced_terms))

    return y0i_ds
#     return preview_term * prod(produced_terms)

In [110]:
random_source_prefix
random_channel_samples = [sample_pY0iX0k(random_source_prefix) for each in range(50)]; random_channel_samples

'⋊.p.ɹ.ɑ.b.ə.b.l'

['⋊.p.g.æ.b.aʊ.b',
 '⋊.m.t.b.ʊ.h.f',
 '⋊.b.ɹ.z.b.ʌ.b',
 '⋊.p.ɹ.ɑ.b.s.b',
 '⋊.s.ɹ.ɑ.ɛ.g.b',
 '⋊.p.eɪ.ɑ.b.ɪ.ɑ',
 '⋊.p.ɹ.t.ŋ.ʌ.b',
 '⋊.p.ɹ.ʒ.t.ɑ.b',
 '⋊.ɹ.ɹ.ɑ.n.p.ʌ',
 '⋊.p.ɹ.ɑ.u.æ.ʃ',
 '⋊.p.ɹ.ɑ.b.eɪ.b',
 '⋊.ʃ.ɹ.ɑ.b.ɪ.b',
 '⋊.ʌ.i.ɑ.b.w.b',
 '⋊.p.ɹ.aɪ.ʊ.j.u',
 '⋊.p.j.ɑ.b.eɪ.ʌ',
 '⋊.p.ʃ.ɑ.ɛ.ʌ.b',
 '⋊.p.ɹ.ɑ.b.ʌ.b',
 '⋊.aɪ.ɑ.ɑ.ɔɪ.ɛ.b',
 '⋊.p.æ.ɑ.b.ʌ.j',
 '⋊.p.ʊ.ɑ.b.ʃ.b',
 '⋊.ɑ.æ.ɑ.b.ʌ.b',
 '⋊.p.ɹ.ɑ.ɪ.ʌ.b',
 '⋊.b.ɹ.t.ð.m.b',
 '⋊.ɹ.ɹ.ʒ.b.ʌ.b',
 '⋊.f.aʊ.ɑ.b.ɹ.b',
 '⋊.aʊ.ɪ.ɑ.ɹ.i.b',
 '⋊.ʊ.ɔɪ.ɑ.b.ʌ.b',
 '⋊.p.ɹ.ɑ.θ.aʊ.b',
 '⋊.p.ɹ.b.b.ɔɪ.b',
 '⋊.p.ɹ.ɑ.h.ð.i',
 '⋊.ɚ.ɹ.ɑ.b.ʌ.ɚ',
 '⋊.ɚ.ɑ.ɛ.b.m.oʊ',
 '⋊.p.ɹ.ɛ.ʒ.ʊ.tʃ',
 '⋊.p.aʊ.ɑ.ʃ.aɪ.ʒ',
 '⋊.p.ɹ.ɑ.b.ʌ.b',
 '⋊.p.ɹ.ɑ.ɪ.z.b',
 '⋊.p.j.k.aɪ.ɛ.b',
 '⋊.p.ɹ.ɑ.b.ʊ.b',
 '⋊.ɪ.ʊ.ɑ.b.ʌ.ŋ',
 '⋊.p.ɹ.ɑ.b.ʒ.b',
 '⋊.θ.ɹ.ɑ.b.ʊ.ʌ',
 '⋊.p.s.u.b.eɪ.b',
 '⋊.n.aʊ.aʊ.b.i.b',
 '⋊.p.ɹ.ɑ.b.ɪ.b',
 '⋊.dʒ.ɹ.p.v.u.b',
 '⋊.p.w.ɹ.b.ʃ.b',
 '⋊.u.p.ɑ.tʃ.ʌ.ʒ',
 '⋊.ɑ.w.ɑ.b.ʃ.b',
 '⋊.eɪ.ɹ.ʃ.b.eɪ.aɪ',
 '⋊.p.ɹ.ɑ.b.ʌ.b']

In [111]:
[sample_pY0iX0k('⋊.k.ɑ.n.t') for each in range(50)]

['⋊.ʊ.ɹ.n',
 '⋊.k.ɑ.s',
 '⋊.k.ɑ.t',
 '⋊.j.ɑ.n',
 '⋊.k.v.ɚ',
 '⋊.z.ɑ.g',
 '⋊.k.ʌ.n',
 '⋊.k.ɪ.f',
 '⋊.k.ʌ.j',
 '⋊.k.ɑ.n',
 '⋊.oʊ.j.ʌ',
 '⋊.n.ɹ.n',
 '⋊.k.ɑ.t',
 '⋊.k.j.n',
 '⋊.k.ʌ.n',
 '⋊.tʃ.æ.i',
 '⋊.k.ɑ.n',
 '⋊.k.ɑ.n',
 '⋊.k.ɑ.n',
 '⋊.k.ɑ.n',
 '⋊.k.f.n',
 '⋊.k.ɑ.n',
 '⋊.k.ɑ.n',
 '⋊.j.ɑ.n',
 '⋊.k.ʒ.b',
 '⋊.aɪ.ɑ.n',
 '⋊.k.ɑ.j',
 '⋊.k.ʌ.n',
 '⋊.k.ɑ.ʃ',
 '⋊.ɚ.ɑ.n',
 '⋊.k.ɑ.n',
 '⋊.k.ʌ.k',
 '⋊.ɹ.ɑ.n',
 '⋊.oʊ.ɑ.n',
 '⋊.l.ʌ.n',
 '⋊.k.ʌ.s',
 '⋊.k.ɑ.s',
 '⋊.k.ɪ.n',
 '⋊.k.ɑ.n',
 '⋊.aɪ.ʌ.n',
 '⋊.l.ɔɪ.w',
 '⋊.k.ɑ.n',
 '⋊.k.ɑ.n',
 '⋊.k.ʌ.dʒ',
 '⋊.k.ɑ.n',
 '⋊.ʃ.ɑ.n',
 '⋊.i.ɑ.n',
 '⋊.k.ɑ.v',
 '⋊.b.h.ɚ',
 '⋊.k.i.m']

## $p(\widehat{X_0^f}|Y_0^{i})$

$p(\widehat{x_0^f}|y_0^{i}) = \frac{p(y_0^{i}|\widehat{x_0^f})p(\widehat{x_0^f})}{p(y_0^{i})}$

In [112]:
def pXhat0fY0i_old(xhat0f, y0i):
    numerator = pXhat0fY0i_unnormalized(xhat0f, y0i)
    denominator = pY0i(y0i)
    if denominator == 0.0:
#         if numerator == 0.0:
#             print('both numerator and denominator are 0')
#         else:
#             print('just the denominator is 0')
        return 0
    return numerator / denominator

(random_source_wordform, random_channel_prefix)
pXhat0fY0i_old(random_source_wordform, random_channel_prefix)

('⋊.g.ɑ.ɹ.n.ɪ.ʃ.⋉', '⋊.ɔɪ.dʒ.ɪ.h.b.h.ɚ')

Exception: |x0f| must ≥ |y0i| + 1

## $\widehat{p}(\widehat{X_0^f}|X_0^i;X_{i+1})$

$p(\widehat{x_0^f}|x_0^i;x_{i+1}) = \sum\limits_{y_0^{i}} p(\widehat{x_0^f}|y_0^{i})p(y_0^{i}|x_0^{i};x_{i+1})$

$\widehat{p}(\widehat{x_0^f}|x_0^i;x_{i+1}) = \frac{1}{m}\sum\limits_{m \text{ sampled } y_0^{i}\text{s from } p(Y_0^{i}|x_0^{i};x_{i+1})} p(\widehat{x_0^f}|y_0^{i})$

In [113]:
# BASELINE = OLD
def phatXhat0fX0k_baseline(xhat0f, x0k, m = 50):
#     y0ks = tuple(sample_pY0kX0k(x0k) for each in range(n))
#     terms = (pXhat0fY0k(xhat0f, y0k) for y0k in y0ks)

#     foo = partial(pXhat0fY0i, xhat0f)
#     def foo(y0i):
#         return pXhat0fY0i(xhat0f, y0i)
#     terms = (foo(y0i) for y0i in (sample_pY0iX0i(x0i) for each in range(n)))
#     terms = Parallel(n_jobs=8, prefer="threads")(delayed(foo)(y0i) for y0i in y0is)
#     terms = Parallel(n_jobs=8, prefer="threads")(delayed(foo)(y0i) for y0i in (sample_pY0iX0i(x0i) for each in range(n)))
#     terms = Parallel(n_jobs=5, verbose=50, batch_size=10, prefer="threads")(delayed(pXhat0fY0i)(xhat0f, y0i) for y0i in (sample_pY0iX0i(x0i) for each in range(n)))
#     terms = Parallel(n_jobs=-1, verbose=50, batch_size=round(n/12), prefer="processes", backend="multiprocessing")(delayed(pXhat0fY0i)(xhat0f, y0i) for y0i in (sample_pY0iX0i(x0i) for each in range(n)))

#     terms = Parallel(n_jobs=-1, batch_size=round(n/12), prefer="processes", backend="multiprocessing")(delayed(pXhat0fY0k_old)(xhat0f, y0k) for y0k in (sample_pY0kX0k(x0k) for each in range(n)))
    terms = Parallel(n_jobs=-1, batch_size='auto', prefer="processes", backend="multiprocessing")(delayed(pXhat0fY0i_old)(xhat0f, y0i) for y0i in (sample_pY0iX0k(x0k) for each in range(m)))
#     terms = (pXhat0fY0k_old(xhat0f, y0k) for y0k in (sample_pY0kX0k(x0k) for each in range(n)))
    s = sum(terms)
    sbar = (1.0 * s) / m
    return sbar

In [114]:
phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 200)

0.0028846549882739626

In [115]:
phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 50)

0.0038308524217430294

In [116]:
Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')

2773

In [118]:
'⋊.k.ɑ.n.t'
len( ds2t('⋊.k.ɑ.n.t') )
len( ds2t('⋊.k.ɑ.n.t') ) - 1
retrieve_CM_for('⋊.k.ɑ.n.t.æ.k.t.⋉', 4)

np.allclose( retrieve_CM_for('⋊.k.ɑ.n.t.æ.k.t.⋉', 4), 
             pY0f_X0ks[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')])

'⋊.k.ɑ.n.t'

5

4

array([[ 0.04390334,  0.01805178,  0.00911065,  0.00901697],
       [ 0.00700978,  0.01374583,  0.00911065,  0.00901697],
       [ 0.02545656,  0.00721312,  0.00911065,  0.00901697],
       [ 0.00700978,  0.00711758,  0.00898997,  0.017121  ],
       [ 0.00682531,  0.00692774,  0.01700733,  0.00866023],
       [ 0.00700978,  0.00721312,  0.00911065,  0.00901697],
       [ 0.00682531,  0.0070233 ,  0.0088709 ,  0.00877968],
       [ 0.00700978,  0.00721312,  0.00911065,  0.00901697],
       [ 0.00682531,  0.00711758,  0.00898997,  0.00889754],
       [ 0.00700978,  0.00721312,  0.00911065,  0.00901697],
       [ 0.0051651 ,  0.00619171,  0.00782054,  0.00774012],
       [ 0.65246265,  0.00711758,  0.00898997,  0.00889754],
       [ 0.02545656,  0.00721312,  0.00911065,  0.00901697],
       [ 0.00700978,  0.00711758,  0.00898997,  0.00889754],
       [ 0.00700978,  0.00721312,  0.6450247 ,  0.00901697],
       [ 0.00700978,  0.01374583,  0.00911065,  0.00901697],
       [ 0.00700978,  0.

True

In [ ]:
#  Preprocessing lexicon channel matrices

In [ ]:
# If the command-line parameter `r` is `False`, then this notebook will only calculate
# $$p(\hat{W}|W = w, C)$$
# rather than
# $$p(\hat{W}|P = p, C)$$ 

# I.e. `r = True` causes source prefixes to be considered where `r = False` causes prefixes to be ignored.

# If `r = False`, then the pre-calculated channel matrices by length and prefix index can be filtered by removing matrices corresponding to proper prefixes.

# Calculation

In [119]:
from random import choice

In [120]:
random_wordform = choice(Ws_t)
random_wordform_length = len(ds2t(random_wordform))
random_wordform_idx = Ws_t.index(random_wordform)
random_wordform, random_wordform_idx, random_wordform_length

('⋊.p.ɹ.ə.l.ɑ.ŋ.⋉', 4925, 8)

In [121]:
type(pW_C)
pW_C.shape
pW_C[:,0].shape

numpy.ndarray

(9172, 106295)

(9172,)

In [122]:
Cs_t[Cs_t.index('a couple of')]

'a couple of'

In [123]:
pX0f = pW_C[:,Cs_t.index('a couple of')]
pX0f_torch = torch.from_numpy(pX0f)

In [124]:
if r:
    CMsByLengthByWordformIndex = pY0f_X0ks
    CMsByLengthByWordformIndex_torch = [torch.from_numpy(each) for each in CMsByLengthByWordformIndex]
else:
#     xCMsByLengthByWordformIndex_torch
    raise Exception('Feature not implemented yet')
#     CMsByLengthByWordformIndex = pY0f_X0ks
#     CMsByLengthByWordformIndex_torch = [torch.from_numpy(each) for each in CMsByLengthByWordformIndex]
#     pass

In [125]:
random_wordform
ds2t(random_wordform)[1:-1]

'⋊.p.ɹ.ə.l.ɑ.ŋ.⋉'

('p', 'ɹ', 'ə', 'l', 'ɑ', 'ŋ')

In [126]:
random_wordform
ds2t(random_wordform)
random_wordform_idx
random_wordform_length
random_wordform_CM = CMsByLengthByWordformIndex_torch[random_wordform_length-3][random_wordform_idx]
'--'
retrieve_CM_for(random_wordform, len(ds2t(random_wordform)[1:-1]))
np.allclose(retrieve_CM_for(random_wordform, len(ds2t(random_wordform)[1:-1])),
            CMsByLengthByWordformIndex[random_wordform_length-3][random_wordform_idx])
'--'
CMsByLengthByWordformIndex_torch[random_wordform_length-3][random_wordform_idx].shape
random_wordform_CM

'⋊.p.ɹ.ə.l.ɑ.ŋ.⋉'

('⋊', 'p', 'ɹ', 'ə', 'l', 'ɑ', 'ŋ', '⋉')

4925

8

'--'

array([[ 0.01110786,  0.01136725,  0.01632438,  0.00904842,  0.02331147,
         0.00700978],
       [ 0.01110786,  0.02166223,  0.01632438,  0.00904842,  0.00781633,
         0.00700978],
       [ 0.01110786,  0.01136725,  0.01632438,  0.02264485,  0.00781633,
         0.00700978],
       [ 0.01110786,  0.01121668,  0.01610816,  0.00892857,  0.01469806,
         0.00682531],
       [ 0.01081555,  0.01091752,  0.01567853,  0.00869043,  0.00750709,
         0.00664084],
       [ 0.01110786,  0.01136725,  0.05542198,  0.0172433 ,  0.00781633,
         0.00700978],
       [ 0.01081555,  0.01106811,  0.01589479,  0.0088103 ,  0.00761064,
         0.00682531],
       [ 0.01110786,  0.01136725,  0.01632438,  0.00904842,  0.00781633,
         0.02545656],
       [ 0.01081555,  0.01121668,  0.01610816,  0.00892857,  0.0077128 ,
         0.04390334],
       [ 0.01110786,  0.01136725,  0.01632438,  0.00904842,  0.00781633,
         0.06235012],
       [ 0.00818474,  0.00975759,  0.01401277,  0.

True

'--'

torch.Size([38, 6])

tensor([[0.0111, 0.0114, 0.0163, 0.0090, 0.0233, 0.0070],
        [0.0111, 0.0217, 0.0163, 0.0090, 0.0078, 0.0070],
        [0.0111, 0.0114, 0.0163, 0.0226, 0.0078, 0.0070],
        [0.0111, 0.0112, 0.0161, 0.0089, 0.0147, 0.0068],
        [0.0108, 0.0109, 0.0157, 0.0087, 0.0075, 0.0066],
        [0.0111, 0.0114, 0.0554, 0.0172, 0.0078, 0.0070],
        [0.0108, 0.0111, 0.0159, 0.0088, 0.0076, 0.0068],
        [0.0111, 0.0114, 0.0163, 0.0090, 0.0078, 0.0255],
        [0.0108, 0.0112, 0.0161, 0.0089, 0.0077, 0.0439],
        [0.0111, 0.0114, 0.0163, 0.0090, 0.0078, 0.0624],
        [0.0082, 0.0098, 0.0140, 0.0078, 0.0067, 0.0070],
        [0.0108, 0.0112, 0.0161, 0.0089, 0.0077, 0.0070],
        [0.0111, 0.0114, 0.0163, 0.6409, 0.0233, 0.0624],
        [0.0111, 0.0214, 0.0161, 0.0089, 0.0147, 0.0068],
        [0.0111, 0.0114, 0.0163, 0.0090, 0.0196, 0.0070],
        [0.0111, 0.0114, 0.0487, 0.0090, 0.0078, 0.0255],
        [0.5665, 0.0112, 0.0161, 0.0089, 0.0077, 0.0068],
        [0.010

## Channel sampler

In [127]:
# if not r:
    def depthSampler2a_t(CM, m=1):
        '''
        Given 
         - a sequence of channel distributions (i.e. a pytorch tensor)
           associated with some prefix x_0^i of the lexicon
         - a number of samples m
        this returns m samples from the channel distribution p(Y_0^i|x_0^i),
        where each 'sample' (i.e. channel string) is a sequence of one hot vectors.
        
        Let l = i+1:
        |depthSampler2a_t(|CM| = (|Y1|, l-1), m)| = (m, l, |Y1|)
        '''
        stack = torch.zeros((m, CM.shape[1], CM.shape[0]))
#         print(f'{stack.dtype}')
        for eachStack in torch.arange(m):
            for i in torch.arange(CM.shape[1]):
                stack[eachStack, i] = torch.distributions.Multinomial(1, CM[:,i]).sample()
        return stack
#         return torch.squeeze(stack)
# else:
#     def depthSampler2a_t(xCM, m=1):
#         stack = torch.zeros((m, xCM.shape[1], xCM.shape[0]))
#         for eachStack in torch.arange(m):
#             for i in torch.arange(xCM.shape[1]):
#                 stack[eachStack, i] = torch.distributions.Multinomial(1, xCM[:,i]).sample()
#         return stack

In [128]:
#FIXME copy what you need from the segmental sequence notebook to interpret these samples

In [129]:
random_wordform
sampled_channel_sequences = depthSampler2a_t(random_wordform_CM, m=2)
sampled_channel_sequences
channelSeqOHs2ds(sampled_channel_sequences.numpy()[0], False)
channelSeqOHs2ds(sampled_channel_sequences.numpy()[0], True)
sampled_channel_sequences.shape

'⋊.p.ɹ.ə.l.ɑ.ŋ.⋉'

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
          0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       

'p.ɹ.ɔɪ.l.ʌ.ɪ'

'⋊.p.ɹ.ɔɪ.l.ʌ.ɪ'

torch.Size([2, 6, 38])

In [130]:
# channelSeqOHs2ds(np.array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
#           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
#           0., 0., 0., 0.]]))

## Sample-parallel posterior estimator

In [131]:
# if not r:
    def pXhat0fX0i_pxt(xhat0f_idx, x0k_CM, m = 50):
        '''
        If ⋊ x_0 x_1 ... x_i x_k denotes
        the produced prefix r of the speaker's intended wordform w,
         where
           x_i = the last fully produced segment
           k = i+1
           x_k = the next segment
           l = i+1 = the length of the produced prefix (not incl. word edges)
        
        then
        
        x0k_CM :: (|Y1|, l) is the associated l-length stack of size |Y1| vectors
         where x0k_CM[:,j] is the distribution p(Y_j|X0^k has been produced)
           
        xhat0f_idx is the index of a wordform w' s.t. 
        
        pXhat0fX0i_pxt(xhat0f_idx, x0k_CM) ≈ p(w'|r)
        '''
        # Computation proceeds in two steps:
        # calculate the denominator p(y0i) for m = 50 sampled y0i's
        # calculate the numerator p(y0i|w')p(w') for m = 50 sampled y0i's
        
        
    #     xhat0f_idx = Wmap[xhat0f]

    #     l = len(ds2t(x0i))
    #     x0k_CM = CMsByPrefixIndex[prefixMap[x0i]]

    #     my_Q_l = CMsByLengthByWordformIndex[l - 2]

        #Collect m samples from p(Y0i|X0k) = p(Y0i|r), 
        # where each sample is an l-length stack of one-hot vectors
        # where each one-hot vector corresponds to a channel symbol:
        Y_prime = depthSampler2a_t(x0k_CM, m)#.float() #:: (m, l, |Y1|)
        l = Y_prime.shape[1]# + 1
        print(f'|Y_prime| = {Y_prime.shape} = (m, l, |Y1|)')

        #Grab ALL the relevant-prefix-length channel matrices 
        # for every segmental wordform in the lexicon:
#         my_Q_l = CMsByLengthByWordformIndex_torch[l - 3]#.float()
        my_Q_l = CMsByLengthByWordformIndex_torch[l - 1]#.float() #:: (|W|, |Y1|, l)
        print(f'|Q_l| = {my_Q_l.shape} = (|W|=n, |Y1|, l)')
    
#         print(f'{Y_prime.dtype}, {my_Q_l.dtype}')
    
        # NORMALIZATION
        V_prime = torch.einsum('mli,kil->mkl', Y_prime, my_Q_l)  # :: (m,n,l)
        print(f"|V'| = {V_prime.shape} = (m, n, l)")
        M_prime = torch.prod(V_prime, 2) # :: (m,n)
        print(f"|M'| = {M_prime.shape} = (m, n)")
        N_prime = torch.matmul(M_prime, pX0f_torch) # :: (m, 1) <- prior probabilities of each of the m sampled channel prefixes
        print(f"|N'| = {N_prime.shape} = (m, )")
        Z_prime = 1.0 / N_prime # :: (m, 1)
        print(f"|Z'| = {Z_prime.shape} = (m, )")

        # NUMERATOR
        L_w = my_Q_l[xhat0f_idx]#.float()
        V_prime_w = torch.einsum('mij,ji->mi',Y_prime, L_w)
        print(f"|V'_w| = {V_prime_w.shape} = (m, l)")
        O_w = torch.prod(V_prime_w, 1) # :: (m,1) likelihoods of each of the m sampled channel prefixes
        print(f"|O_w| = {O_w.shape} = (m, )")
        U_w = pX0f_torch[xhat0f_idx] * O_w ## :: (m,1) joint probabilities of xhat0f with each of the m sampled channel prefixes
        print(f"|U_w| = {U_w.shape} = (m, )")
        
        return torch.dot(Z_prime, U_w) / m

    #     return torch.dot( 1.0 / torch.matmul(torch.prod(torch.einsum('mli,kil->mkl', Y_prime, my_Q_l), 2), pX0f_torch) , 
    #        pX0f_torch[xhat0f_idx] * torch.prod(torch.einsum('mij,ji->mi',Y_prime, L_w), 1) ) / m
# else:
#     def pXhat0fX0k_pxt(xhat0f_idx, x0i_xCM, m = 50):
#     #     xhat0f_idx = Wmap[xhat0f]

#     #     l = len(ds2t(x0i))
#     #     x0i_xCM = xCMsByPrefixIndex[prefixMap[x0i]]

#     #     my_Q_l = xCMsByLengthByWordformIndex[l - 2]

#         Y_prime = depthSampler2a_t(x0i_xCM, m)#.float()
#         l = Y_prime.shape[1] + 1

#         my_Q_l = xCMsByLengthByWordformIndex_torch[l - 3]#.float()

#         # NORMALIZATION
#         V_prime = torch.einsum('mli,kil->mkl', Y_prime, my_Q_l)  # :: (m,n,l)
#         M_prime = torch.prod(V_prime, 2) # :: (m,n)
#         N_prime = torch.matmul(M_prime, pX0f_torch) # :: (m, 1) <- prior probabilities of each of the m sampled channel prefixes
#         Z_prime = 1.0 / N_prime # :: (m, 1)

#         # NUMERATOR
#         L_w = my_Q_l[xhat0f_idx]#.float()
#         V_prime_w = torch.einsum('mij,ji->mi',Y_prime, L_w)
#         O_w = torch.prod(V_prime_w, 1) # :: (m,1) likelihoods of each of the m sampled channel prefixes
#         U_w = pX0f_torch[xhat0f_idx] * O_w ## :: (m,1) joint probabilities of xhat0f with each of the m sampled channel prefixes

#         return torch.dot(Z_prime, U_w) / m

#     #     return torch.dot( 1.0 / torch.matmul(torch.prod(torch.einsum('mli,kil->mkl', Y_prime, my_Q_l), 2), pX0f_torch) , 
#     #        pX0f_torch[xhat0f_idx] * torch.prod(torch.einsum('mij,ji->mi',Y_prime, L_w), 1) ) / m

In [132]:
random_wordform
random_wordform_length
retrieve_CM_for(random_wordform, random_wordform_length-2).shape
np.allclose(retrieve_CM_for(random_wordform, random_wordform_length-2),
            random_wordform_CM.numpy())
# random_wordform_CM = CMsByLengthByWordformIndex_torch[random_wordform_length-3][random_wordform_idx]
random_wordform_idx
' '
Y_PRIME = depthSampler2a_t(random_wordform_CM, m=2); Y_PRIME.shape
L = Y_PRIME.shape[1]; L
pXhat0fX0i_pxt(random_wordform_idx, random_wordform_CM)
# CMsByLengthByWordformIndex_torch[L - 3].shape # <<< this should either be (|Ws| x |Y1s| x ?)

'⋊.p.ɹ.ə.l.ɑ.ŋ.⋉'

8

(38, 6)

True

4925

' '

torch.Size([2, 6, 38])

6

|Y_prime| = torch.Size([50, 6, 38]) = (m, l, |Y1|)
|Q_l| = torch.Size([9172, 38, 6]) = (|W|=n, |Y1|, l)
|V'| = torch.Size([50, 9172, 6]) = (m, n, l)
|M'| = torch.Size([50, 9172]) = (m, n)
|N'| = torch.Size([50]) = (m, )
|Z'| = torch.Size([50]) = (m, )
|V'_w| = torch.Size([50, 6]) = (m, l)
|O_w| = torch.Size([50]) = (m, )
|U_w| = torch.Size([50]) = (m, )


tensor(0.1974)

In [133]:
len(Ws_t)

9172

In [134]:
pXhat0fX0i_pxt(random_wordform_idx, 
               random_wordform_CM)

|Y_prime| = torch.Size([50, 6, 38]) = (m, l, |Y1|)
|Q_l| = torch.Size([9172, 38, 6]) = (|W|=n, |Y1|, l)
|V'| = torch.Size([50, 9172, 6]) = (m, n, l)
|M'| = torch.Size([50, 9172]) = (m, n)
|N'| = torch.Size([50]) = (m, )
|Z'| = torch.Size([50]) = (m, )
|V'_w| = torch.Size([50, 6]) = (m, l)
|O_w| = torch.Size([50]) = (m, )
|U_w| = torch.Size([50]) = (m, )


tensor(0.1949)

In [135]:
Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')

'⋊.k.ɑ.n.t'
len( ds2t('⋊.k.ɑ.n.t') )
len( ds2t('⋊.k.ɑ.n.t') ) - 1
retrieve_CM_for('⋊.k.ɑ.n.t.æ.k.t.⋉', 4)

np.allclose( retrieve_CM_for('⋊.k.ɑ.n.t.æ.k.t.⋉', 4), 
             CMsByLengthByWordformIndex[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')])

2773

'⋊.k.ɑ.n.t'

5

4

array([[ 0.04390334,  0.01805178,  0.00911065,  0.00901697],
       [ 0.00700978,  0.01374583,  0.00911065,  0.00901697],
       [ 0.02545656,  0.00721312,  0.00911065,  0.00901697],
       [ 0.00700978,  0.00711758,  0.00898997,  0.017121  ],
       [ 0.00682531,  0.00692774,  0.01700733,  0.00866023],
       [ 0.00700978,  0.00721312,  0.00911065,  0.00901697],
       [ 0.00682531,  0.0070233 ,  0.0088709 ,  0.00877968],
       [ 0.00700978,  0.00721312,  0.00911065,  0.00901697],
       [ 0.00682531,  0.00711758,  0.00898997,  0.00889754],
       [ 0.00700978,  0.00721312,  0.00911065,  0.00901697],
       [ 0.0051651 ,  0.00619171,  0.00782054,  0.00774012],
       [ 0.65246265,  0.00711758,  0.00898997,  0.00889754],
       [ 0.02545656,  0.00721312,  0.00911065,  0.00901697],
       [ 0.00700978,  0.00711758,  0.00898997,  0.00889754],
       [ 0.00700978,  0.00721312,  0.6450247 ,  0.00901697],
       [ 0.00700978,  0.01374583,  0.00911065,  0.00901697],
       [ 0.00700978,  0.

True

In [ ]:
pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
              CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
              m = 50)

In [ ]:
sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 10) for each in range(10)])/10
sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 50) for each in range(10)])/10
sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 100) for each in range(10)])/10
phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 250)
phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 500)
phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 1000)
phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 2500)
phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 5000)
phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 10000)

0.002809647113163415

0.0032816590277715712

In [ ]:
pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
              CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
              m = 50)
pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
              CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
              m = 100)
pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
              CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
              m = 200)
pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
              CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
              m = 500)
pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
              CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
              m = 1000)
pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
              CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
              m = 2000)
pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
              CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
              m = 5000)